# Introduction
In this project, I create an AI chatbot about airlines. The chatbot will give information about airline tickets and available chairs.

In [1]:
# Library
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
# Checking API key for using the AI model from Openai
load_dotenv(override = True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [19]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [7]:
# I create the tools function for ai model.
airplan_chair_statu = {'1' : 'Empty',
                      '2' : 'Empty',
                      '3' : 'Full',
                      '4' : 'Full',
                      '6' : 'Empty',
                      '7' : 'Empty',
                      '8' : 'Empty',
                      '9' : 'Empty',
                      '10' : 'Full',
                      '11' : 'Full',
                      '12' : 'Empty',
                      '13' : 'Empty'}

tickets_hour = {'paris': [{'hour': '18.00', 'price': '$599'},
                    {'hour': '22.00', 'price': '$399'}],
                'london': [{'hour': '12.00', 'price': '$499'}],
                'istanbul': [{'hour': '15.00', 'price': '$449'},
                      {'hour': '18.00', 'price': '$599'},
                      {'hour': '21.00', 'price': '$499'}],
                'berlin': [{'hour': '11.00', 'price': '$299'},
                    {'hour': '15.00', 'price': '$499'},
                    {'hour': '18.00', 'price': '$799'}]
               }

ticket_place = {"london": "Not Full", "paris": "Not Full", "istanbul": "Not Full", "berlin": "Not Full", "tokyo": "Full", "california": "Full", "pekin": "Full"}

def get_ticket_place(destination_city):
    print(f"Select the destination city is {destination_city}")
    city = destination_city.lower()
    return ticket_place.get(city, "Unknown")

def get_tickets_hours(destination_city):
    print(f"Select the destination city is {destination_city}")
    city = destination_city.lower()
    return tickets_hour.get(city, 'Unknown')

def get_chair(number_of_chair):
    print(f"Select the number of chair {number_of_chair}")
    return airplan_chair_statu.get(number_of_chair, 'Unknown')

In [8]:
get_ticket_place('Istanbul')

Select the destination city is Istanbul


'Not Full'

In [9]:
get_ticket_place('Tokyo')

Select the destination city is Tokyo


'Full'

In [10]:
get_tickets_hours('london')

Select the destination city is london


[{'hour': '12.00', 'price': '$499'}]

In [11]:
get_tickets_hours('istanbul')

Select the destination city is istanbul


[{'hour': '15.00', 'price': '$449'},
 {'hour': '18.00', 'price': '$599'},
 {'hour': '21.00', 'price': '$499'}]

In [12]:
get_chair('1')

Select the number of chair 1


'Empty'

In [13]:
get_chair('3')

Select the number of chair 3


'Full'

In [14]:
# I prepare the tool funtion for openai ai model. If the model use funtion you have to prepare the functions.
ticket_place_function = {
    'name': 'get_ticket_place',
    'description': "Check if there are available tickets for the destination city. Use this whenever a customer asks about ticket availability, for example when they ask 'Is there an available ticket to this city?'",
    'parameters': {
        'type' : 'object',
        'properties': {
            'destination_city': {
                'type': 'string',
                'description' : 'The city that the customer wants to check for ticket availability.',
            },
        },
        'required': ['destination_city'],
        #'additionalProperties': False
    }
}

tickets_hour_function = {
    'name': 'get_tickets_hours',
    'description': "Retrieve available ticket hours and prices for a given destination city. Use this whenever a customer asks about ticket times and costs, for example when they ask 'What time are the available flights to this city and how much do they cost?'",
    'parameters': {
        'type': 'object',
        'properties': {
            'destination_city': {
                'type' : 'string',
                'description': "The city for which the customer wants to check available ticket hours and prices.",
            },
        },
        'required': ['destination_city'],
        #'additionalProperties': False
    }
}

chair_status_function = {
    'name': 'get_chair',
    'description': "Check whether a specific airplane seat is available or full. Use this whenever a customer asks about the status of a specific seat.",
    'parameters': {
        'type' : 'object',
        'properties': {
            'number_of_chair' : {
                'type': 'string',
                'description': "The seat number the customer wants to check.",
            },
        },
        'required': ['number_of_chair']
    }
}

In [15]:
tools = [{'type': 'function', 'function': ticket_place_function},
        {'type': 'function', 'function': tickets_hour_function},
        {'type': 'function', 'function': chair_status_function}]

In [17]:
# The functions below are prepared for use by the AI ​​model.
def handle_tool_call_v2(message):
    tool_call = message.tool_calls[0]
    function_name = tool_call.function.name
    arguments = json.loads(tool_call.function.arguments)

    if function_name == "get_ticket_place":
        city = arguments.get('destination_city')
        available_tickets = get_ticket_place(city)
        response_content = {'destination_city': city, 'available_tickets': available_tickets}

        response = {
            'role': 'tool',
            'content': json.dumps(response_content),
            'tool_call_id': tool_call.id
        }

        return response, city

    elif function_name == "get_tickets_hours":
        city = arguments.get('destination_city')
        tickets_hours_and_price = get_tickets_hours(city)
        response_content = {'destination_city': city, 'tickets_hours_and_price': tickets_hours_and_price}

        response = {
            'role': 'tool',
            'content': json.dumps(response_content),
            'tool_call_id': tool_call.id
        }

        return response, city

    elif function_name == "get_chair":
        number_of_chair = arguments.get('number_of_chair')
        chair_statu = get_chair(number_of_chair)
        response_content = {'destination_city': number_of_chair, 'tickets_hours_and_price': chair_statu}

        response = {
            'role': 'tool',
            'content': json.dumps(response_content),
            'tool_call_id': tool_call.id
        }

        return response, number_of_chair

In [16]:
# Chat function
def chat_v2(message, history):
    messages = [{'role': 'system', 'content': system_message}] + history + [{'role': 'user', 'content': message}]
    response = openai.chat.completions.create(
        model = 'gpt-4o-mini',
        messages = messages,
        tools = tools,
    )

    if response.choices[0].finish_reason == 'tool_calls':
        message = response.choices[0].message
        response, _ = handle_tool_call_v2(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(
            model = 'gpt-4o-mini',
            messages = messages
        )
        
    return response.choices[0].message.content

In [35]:
# If You want to see chatbot you can use below code.
#chatbot = gr.ChatInterface(fn = chat_v2, type = 'messages')
#app = chatbot.launch()

# CONCLUSION
In this project, I used the GPT-4o-mini model at the Openai. I prepared the tools for my models and I used the gradio library to see the chat. This project is about selling tickets for an airline company. I want to see how to prepare AI model and tools for AI model. I hope my project was useful for you. Thank you for taking a look at my project. I will continue to share projects. If you want to be informed in advance, you can follow me from the links below.

[LinkedIn](https://www.linkedin.com/in/ihsancenkiz/)<br>
[GitHub](https://github.com/ihsncnkz)<br>
[Kaggle](https://www.kaggle.com/ihsncnkz)<br>